In [15]:
import pandas as pd
import numpy as np
import re
import json


# Exploring textual info (labels and addresses)


In [16]:
def is_label_in_string(label, i):
    return len(re.findall(fr'\b{str(label)}\b', i))

In [17]:
df_labels = pd.read_csv('datasets/dataset_labels.csv', sep=';')
print('Unique labels:', df_labels.shape[0])
print('Label dataset:')
df_labels.head(5)

Unique labels: 27260
Label dataset:


,id,label
0,0,"Lauenhain (Kreis Hainichen), Fahnenträger eine..."
1,1,Fenster (Bauelement)
2,2,"Kassel, Infanteriekasernen an der Königsstraße..."
3,3,"Waldeck, Domäne, Wohnhaus, Umbauentwurf, Aufrisse"
4,4,"Kassel, Opernhaus (Altes Hoftheater), Aborte i..."


In [18]:
df_outdoor = pd.read_csv('datasets/dataset_MVP_clustered.csv', sep=';')
df_outdoor = df_outdoor[df_outdoor['cluster_type'] == 'outdoor']
print('Size:', df_outdoor.shape)
print('Outdoor images dataset:')
df_outdoor.head(5)

Size: (6796, 7)
Outdoor images dataset:


,link,id_objects,labels,file_name,id,cluster,cluster_type
4,http://www.bildindex.de/bilder/d/ae00001b10,['3899'],[3257],scn-ae00001b10.jpg,ae00001b10,1,outdoor
16,http://www.bildindex.de/bilder/d/ae00002e14,['4006'],[3243],scn-ae00002e14.jpg,ae00002e14,1,outdoor
19,http://www.bildindex.de/bilder/d/ae00003a09,['3918'],[3274],scn-ae00003a09.jpg,ae00003a09,1,outdoor
21,http://www.bildindex.de/bilder/d/ae00003a12,['123'],[107],scn-ae00003a12.jpg,ae00003a12,1,outdoor
22,http://www.bildindex.de/bilder/d/ae00003a13,['453'],[392],scn-ae00003a13.jpg,ae00003a13,74,outdoor


Listing labels related to outdoor images:

In [19]:
outdoor_cluster_ids = df_outdoor['cluster'].unique()
outdoor_labels = []

for i, x in enumerate(df_outdoor.groupby('cluster').labels.unique()):
    print('ID:', outdoor_cluster_ids[i])
    
    # Convert lists of strings to one merged list of integers (ids of labels)
    labels = [int(element) for l in x for element in l[1:-1].split(', ')]
    labels = list(set(labels))
    # Convert ids to textual labels
    labels = [(label, df_labels[df_labels['id'] == label]['label'].values[0]) for label in labels]
    # Sort them
    labels.sort(key=lambda x: x[1])

    print('Labels:')
    for label in labels:
        print(' -', label)
    print()

    outdoor_labels.append(labels)

ID: 1
Labels:
 - (19493, '50 Jahre danach (9.10.-13.11.1988) von Clemens Mitscher und Richard Stumm')
 - (8212, 'Ansicht der Stadt Marburg')
 - (16243, 'Aufriß der Klosterkirche in Daphni (Attika), Südeseite')
 - (4338, 'Ausbau einer Kasematte')
 - (4341, 'Ausbau einer Kasematte Ausbau einer Kasematte/Ausbau einer Kasematte 3')
 - (22845, 'Ausfahrender Zug aus dem Anhalter Bahnhof in Berlin mit zum Abschied Winkenden')
 - (15503, 'Ausstellung')
 - (19991, 'Ausstellung: "Bildhauer im Schloß" des Marburger Kunstvereins (4.7.-29.8.1982)')
 - (19916, 'Ausstellung?')
 - (24061, 'Ausstellungsraum')
 - (9642, 'Bau der Halle des Schlesischen Bahnhofs (Niederschlesisch-Märkischer Bahnhof) in Berlin-Friedrichshain')
 - (23557, 'Bau der alten Bahnhofshalle des Bahnhofs Friedrichstraße in Berlin-Mitte')
 - (12417, 'Bauernhaus in Hemmendorf bei Lübeck')
 - (15122, 'Bildnis Heloise Heffter als Kind')
 - (3977, 'Brunnen')
 - (2664, 'Burg Lauenstein, Erker im Jagdsaal')
 - (2673, 'Burg Lauenstein, Frä

Listing all those pictures, which include `query` in label. Query can be changed


In [21]:
k = 0
marburg_labels_ids = []
marburg_pictures = []

query = 'Kunstgeb'

for i in outdoor_labels:
    for label in i:
        if query in label[1]:
            marburg_labels_ids.append(label[0])
            print(label)
            k += 1

print()
for j in marburg_labels_ids:
    for i in df_outdoor.values:
        if is_label_in_string(str(j), i[2]):
            marburg_pictures.append(i[0])
            # print(i[0])

print(f'{query} outdoor pictures:', len(marburg_pictures))
marburg_pictures[:10]

(12092, 'Kunstgebäude')
(12092, 'Kunstgebäude')
(12100, 'Kunstgebäude Kunstgebäude/Innenhof')
(12093, 'Kunstgebäude Kunstgebäude/Kunstmuseum Marburg')
(12092, 'Kunstgebäude')
(12101, 'Kunstgebäude Kunstgebäude/Garten Ostseite')
(12100, 'Kunstgebäude Kunstgebäude/Innenhof')
(12092, 'Kunstgebäude')
(12092, 'Kunstgebäude')
(12092, 'Kunstgebäude')
(12101, 'Kunstgebäude Kunstgebäude/Garten Ostseite')
(12100, 'Kunstgebäude Kunstgebäude/Innenhof')
(12092, 'Kunstgebäude')

Kunstgeb outdoor pictures: 829


['http://www.bildindex.de/bilder/d/fm1351620',
 'http://www.bildindex.de/bilder/d/fm1351621',
 'http://www.bildindex.de/bilder/d/fm1351622',
 'http://www.bildindex.de/bilder/d/fm1351623',
 'http://www.bildindex.de/bilder/d/fm141330',
 'http://www.bildindex.de/bilder/d/fm15243',
 'http://www.bildindex.de/bilder/d/fm15245',
 'http://www.bildindex.de/bilder/d/fm193412',
 'http://www.bildindex.de/bilder/d/fm21211',
 'http://www.bildindex.de/bilder/d/fm21212']

Searching kinda tockens from addresses in order to later find them in labels. First extract addresses

In [7]:
with open('datasets/buildings_dict.json') as f:
    addresses = json.load(f)

# List of tockens / queries for later search
queries = []

for area in addresses:
    # Adding area of camouses to query
    for campus in addresses[area]:
        for i in addresses[area][campus]:
            if i['address'] != 'Renthof':
                queries.append(i['address'])

# Adding some specific queries
_ = ['Institutsgebäude', 'Kunstgebäude','Dokumentationszentrum','Bibliothek','Mensa',
     'Institut','Hörsaalgebäude', 
    #  'Universität',
       'Sporthalle', 'Stadium', 'Stadion', 'Hörsaal', 
     'Universitätsstadion', 'Universitätsbibliothek',
    'Universitätskirche',
    'Philipps-Universität',
    'Universitäts-Bibliothek',
    'Marburg University',
    'Seminargebäude',
    'Universität in Marburg',
    'Universität Marburg',
    'Universitätsgebäudes in Marburg',
    'Universitäts-Turnhalle']

for i in _:
    queries.append(i)
    
queries = list(set(queries))

print('Queries:', queries)

Queries: ['Wilhelm-Röpke-Straße 4', 'Conradistraße 3a', 'Biegenstraße 12', 'Mensa', 'Biegenstraße 14', 'Kunstgebäude', 'Marbacher Weg 10', 'Hans-Meerwein-Straße 6', 'Mainzer Gasse 33', 'Jahnstraße 12', 'Universitätsstraße 25', 'Philipps-Universität', 'Marbacher Weg 8', 'Marbacher Weg 6', 'Am Plan 1', 'Ketzerbach 63', 'Schloß 3', 'Karl-von-Frisch-Straße 2', 'Karl-von-Frisch-Straße 8', 'Universität in Marburg', 'Karl-von-Frisch-Straße 18', 'Conradistraße 5', 'Hans-Meerwein-Straße 3', 'Universitätsstraße 24', 'Robert-Koch-Straße 4', 'Deutschhausstraße 17a', 'Hans-Meerwein-Straße 8', 'Karl-von-Frisch-Straße 6', 'Deutschhausstraße 17', 'Schulstraße 12', 'Hans-Meerwein-Straße 2a', 'Deutschhausstraße 3', 'Barfüßerstraße 1a', 'Deutschhausstraße 9', 'Karl-von-Frisch-Straße 16', 'Karl-von-Frisch-Straße 14', 'Institut', 'Schloß 4', 'Rudolf-Bultmann-Straße 4b', 'Schloß 1', 'Pilgrimstein 12', 'Gutenbergstraße 18', 'Bahnhofstraße 7', 'Deutschhausstraße 17b', 'Universitäts-Turnhalle', 'Karl-von-Frisc

In [8]:
uni_labels_ids = []
uni_pictures = []

for i in outdoor_labels:
    for label in i:
        for q in queries:
            if q in label[1]:
                uni_labels_ids.append(label[0])

uni_labels_ids = list(set(uni_labels_ids))

for j in uni_labels_ids:
    for i in df_outdoor.values:
        if is_label_in_string(str(j), i[2]):
            uni_pictures.append(i[0])


uni_pictures = list(set(uni_pictures))  

print('Uni outdoor picures:', len(uni_pictures))

Uni outdoor picures: 578


In [9]:
# What if we extrapolate this result to all objects with such labels?
df = pd.read_csv('datasets/dataset_MVP_clustered.csv', sep=';')

# uni_labels_ids = []
# uni_pictures = []

# for i in outdoor_labels:
#     for label in i:
#         for q in queries:
#             if q in label[1]:
#                 uni_labels_ids.append(label[0])

# uni_labels_ids = list(set(uni_labels_ids))

for j in uni_labels_ids:
    for i in df.values:
        if is_label_in_string(str(j), i[2]):
            uni_pictures.append(i[0])


uni_pictures = list(set(uni_pictures))  

print('Uni pictures:', len(uni_pictures))

Uni pictures: 1168


In [10]:
df.shape

(36981, 7)

In [11]:
print('Related labels:')
for i in uni_labels_ids:
    print(i, df_labels[df_labels['id'] == i]['label'].values[0])

Related labels:
19461 Ehemaliges Institut für Nuklearforschung
6667 Philipp I., genannt der Großmütige (Gründer der Philipps-Universität)
3343 Naturwissenschaftliche Institute
18448 Geisteswissenschaftliche Institute der Philipps-Universität
26257 Hörsaalgebäude Lahnberge
5777 Olympischer Wettkampf: 50 km Gehen (Waldschulallee, Stadion Eichkamp in Berlin-Charlottenburg)
274 Universitäts-Turnhalle
9108 Marburg University Research Academy
7581 Alte Stadtschule & Institut für Gesundheitserziehung
12323 Bürgerwiese (heute Gelände der Universitätsbibliothek): Nationalsozialistische Veranstaltung, wahrscheinlich Wahlkundgebung der NSDAP am 20.04.1932
22309 Universitätsstadion
19880 Nuklearmedizinisches Institut & Ehemaliges Chemisches Institut
7082 Neues Physikalisches Institut der Universität
13995 Pläne zum Neubau des Universitätsgebäudes in Marburg Pläne zum Neubau des Universitätsgebäudes in Marburg/Aufriß des westlichen Teils des Südflügels
3375 Pharmazeutisch- Chemisches Institut?
2564

In [13]:
'Kunst'

,id,label
0,0,"Lauenhain (Kreis Hainichen), Fahnenträger eine..."
1,1,Fenster (Bauelement)
2,2,"Kassel, Infanteriekasernen an der Königsstraße..."
3,3,"Waldeck, Domäne, Wohnhaus, Umbauentwurf, Aufrisse"
4,4,"Kassel, Opernhaus (Altes Hoftheater), Aborte i..."
...,...,...
27255,27255,"Kassel, Garnisonkirche, Bauaufnahme, Serie Kas..."
27256,27256,"Kassel, Garnisonkirche, Bauaufnahme, Serie Kas..."
27257,27257,"Kassel, Garnisonkirche, Bauaufnahme, Serie Kas..."
27258,27258,"Kassel-Wilhelmshöhe, Hofgärtnerhaus, Bauaufnah..."


In [37]:
# Search links by labels:
label, is_name_printed = 25648, False
_ = []

for j in uni_labels_ids:
    # for i in df_outdoor.values:
    for i in df.values:
        if len(re.findall(fr'\b{str(label)}\b', i[2])):
            if not is_name_printed:
                print('Label:', df_labels[df_labels['id'] == label]['label'].values[0])
                is_name_printed = True
            _.append(i[0])

_ = list(set(_))
_

Label: Physiologisches Institut


['http://www.bildindex.de/bilder/d/fmd469023',
 'http://www.bildindex.de/bilder/d/fmd469042',
 'http://www.bildindex.de/bilder/d/fmd469183',
 'http://www.bildindex.de/bilder/d/fm14741',
 'http://www.bildindex.de/bilder/d/fm810340',
 'http://www.bildindex.de/bilder/d/fmd469185',
 'http://www.bildindex.de/bilder/d/fmd469182',
 'http://www.bildindex.de/bilder/d/fmd469178',
 'http://www.bildindex.de/bilder/d/fmd469026',
 'http://www.bildindex.de/bilder/d/fmd469040',
 'http://www.bildindex.de/bilder/d/fmd469041',
 'http://www.bildindex.de/bilder/d/fmd469180',
 'http://www.bildindex.de/bilder/d/fmd469187',
 'http://www.bildindex.de/bilder/d/fmd469175',
 'http://www.bildindex.de/bilder/d/fmc428674',
 'http://www.bildindex.de/bilder/d/fmd469025',
 'http://www.bildindex.de/bilder/d/fmd469031',
 'http://www.bildindex.de/bilder/d/fmd469039',
 'http://www.bildindex.de/bilder/d/fmd469021',
 'http://www.bildindex.de/bilder/d/fmd469019',
 'http://www.bildindex.de/bilder/d/fmd469184',
 'http://www.bil

**Conclusion**. We could obtain only 777 pictures by roughly searching for Uni addresses among labels. Obtained building are:  
- Mensa, 
- Bibliothek, 
- Kunstgebäude
- Hörsaalgebäude Lahnberge
- Hörsaalgebäude & Auditorium maximum
- some institutes  
  
At least we know, that we can have storylines for these buildings

## Exploring NLP results
Soroush applied his previous NLP model to dataset `outdoor_images.csv`. We are going to explore and try to evaluate results

In [16]:
# df = pd.read_csv('OutDoor/top_5_similar_buildings.csv', sep=';')
# # df = pd.read_csv('datasets/outdoor_images.csv', sep=';')
# df.head()